In [66]:
import tensorflow as tf
import tensorflow.keras as keras
import random
import os

In [67]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  for gpu in gpus:
    try:
      print("Enabling GPU: ", gpu)
      tf.config.experimental.set_visible_devices(gpu, 'GPU')
      tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
      print(e)

Enabling GPU:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Visible devices cannot be modified after being initialized
Enabling GPU:  PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [68]:
val_size = 0.3
rand_seed = random.randint(0, 2**32 - 1)
img_dim = 96
batch_size = 8

In [69]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "training"),
  validation_split = val_size,
  subset = "training",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size,
  shuffle = True)

val_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "training"),
  validation_split = val_size,
  subset = "validation",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size,
  shuffle = True)

Found 1970 files belonging to 2 classes.
Using 1379 files for training.
Found 1970 files belonging to 2 classes.
Using 591 files for validation.


In [70]:
image_input = keras.Input(shape = (img_dim, img_dim, 3))
x = keras.layers.Rescaling(1.0 / 255)(image_input)
x = keras.layers.Conv2D(128, kernel_size = (3, 3), activation = 'relu')(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.Conv2D(256, kernel_size = (3, 3), activation = 'relu')(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.Dropout(0.01)(x)
feature_output = keras.layers.Flatten()(x)
x = keras.layers.Dense(2048, activation = 'relu')(feature_output)
classification_output = keras.layers.Dense(2)(x)

model = keras.models.Model(image_input, classification_output)

In [71]:
opt = keras.optimizers.Adagrad(learning_rate = 0.01)
model.compile(optimizer = opt,
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_7 (Rescaling)         │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 94, 94, 128)    │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 47, 47, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 45, 45, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 22, 22, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 22, 22, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 123904)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 2048)           │   253,757,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 254,060,290 (969.16 MB)

 Trainable params: 254,060,290 (969.16 MB)

 Non-trainable params: 0 (0.00 B)

In [72]:
class_names = train_ds.class_names
AUTOTUNE = tf.data.AUTOTUNE
epochs = 16

In [73]:
hist = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs, 
  callbacks = []
)

Epoch 1/16


2025-02-09 07:59:34.891021: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_487', 8 bytes spill stores, 8 bytes spill loads



173/173 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.6616 - loss: 0.6401 - val_accuracy: 0.6616 - val_loss: 0.5529
Epoch 2/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9499 - loss: 0.1577 - val_accuracy: 0.9763 - val_loss: 0.0693
Epoch 3/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9719 - loss: 0.0820 - val_accuracy: 0.9848 - val_loss: 0.0665
Epoch 4/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9860 - loss: 0.0453 - val_accuracy: 0.9560 - val_loss: 0.1010
Epoch 5/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9840 - loss: 0.0430 - val_accuracy: 0.9848 - val_loss: 0.0435
Epoch 6/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9947 - loss: 0.0243 - val_accuracy: 0.9831 - val_loss: 0.0432
Epoch 7/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9959 - loss: 0.0171 - val_accuracy: 0.9797 - val_loss: 0.0605
Epoch 8/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9989 - loss: 0.0156 - val_accuracy: 0.986

In [74]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "test"),
  shuffle = False,
  image_size = (img_dim, img_dim)
)

Found 30 files belonging to 2 classes.


In [75]:
model.evaluate(test_ds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 1.0000 - loss: 0.0280


[0.027995234355330467, 1.0]